In [ ]:
import matplotlib.pyplot as plt, netCDF4 as nc, numpy as np
import seaborn as sns
import pandas as pd
import os
import cv2
from skimage.io import imread
from skimage.feature import canny
from skimage.measure import regionprops,label
from collections import Counter


# Introduction
In this notebook, our objective is to employ rainfall and sea level pressure data to identify and track cyclones that occurred during the year 2019. We have at our disposal 3-hourly precipitation data and sea level pressure data, both of which are collected within the geographical bounds of latitude 0° to 30° and longitude 100° to 160°, with a specific focus on the Southeast Asia region. By utilizing these datasets, we aim to identify and monitor cyclonic systems that developed and traversed this region throughout the year 2019. This analysis serves as a valuable tool for understanding cyclone behavior and can aid in the prediction and mitigation of cyclone-related hazards in Southeast Asia.

# Data Loading

Our dataset contains two critical components: 3-hourly rainfall data and hourly sea-level pressure data for the entire year 2019. We separated these datasets into separate variables to simplify our data management. The files containing rainfall data are stored in one variable, while the files containing sea-level pressure data are stored in the other.








In [ ]:
precip_dir = "/kaggle/input/trmm-precipitation/Precipitation/Hourly/2019/"
precip_files = sorted(os.listdir(precip_dir))


In [ ]:
slp_dir = "/kaggle/input/trmm-precipitation/SLP/2019/"
slp_files = sorted(os.listdir(slp_dir))


# Plotting the rainfall and Pressure contours 

In this section, we focus on visualizing the key components of our meteorological datasets: rainfall and sea-level pressure. To provide a clear and intuitive representation of these data, we create contour plots that reveal spatial variations and patterns. These plots aid our understanding of weather conditions and are essential for tracking cyclonic activity.

In [ ]:
start_longitude = 100
end_longitude = 160
start_latitude = 0
end_latitude = 30

In [ ]:
def land_sea_mask(lon_min=-179, lon_max=179, lat_min=-49, lat_max=49):
    # Read the netCDF file
    ls_mask = nc.Dataset("/kaggle/input/trmm-precipitation/mask.nc4")

    # Get the variables from the netCDF file
    lon = ls_mask.variables["lon"][:]
    lat = ls_mask.variables["lat"][:]
    landseamask = ls_mask.variables["landseamask"][:]

    # Convert land-sea mask to binary representation (land=0, sea=1)
    landseamask_binary = np.where(landseamask == 100, 1, 0)


    # Find the corresponding indices for the specified latitude and longitude range
    lon_indices = np.where((lon >= lon_min) & (lon <= lon_max))[0]
    lat_indices = np.where((lat >= lat_min) & (lat <= lat_max))[0]

    # Subset the land-sea mask and corresponding latitude and longitude arrays
    landseamask_subset = landseamask_binary[lon_indices[0]:lon_indices[-1]+1, lat_indices[0]:lat_indices[-1]+1]
    
    return landseamask_subset


In [ ]:
def plot_precipitation(precip_file, lon_min=-179, lon_max=179, lat_min=-49, lat_max=49):
    
    fn = precip_dir + precip_file
    ds = nc.Dataset(fn)
    precipitation = ds["precipitation"][:]

    lat = ds["nlat"][:].data
    lon = ds["nlon"][:].data

    min_lon_index = np.abs(lon_min - lon).argmin()
    max_lon_index = np.abs(lon_max - lon).argmin()
    min_lat_index = np.abs(lat_min - lat).argmin()
    max_lat_index = np.abs(lat_max - lat).argmin()

    precipitation_new = precipitation[min_lon_index: max_lon_index, min_lat_index:max_lat_index]

    plt.figure(figsize=(12, 4))
    levels = np.linspace(0, np.max(precipitation_new), 20)

    lon_subset = lon[min_lon_index: max_lon_index]
    lat_subset = lat[min_lat_index: max_lat_index]
    
    landseamask_binary = land_sea_mask(lon_min, lon_max, lat_min, lat_max)
        
    plt.contourf(lon_subset, lat_subset, landseamask_binary.T, levels=[-0.5, 0.5, 1.5], colors=["papayawhip", "lightskyblue"])
    plt.contourf(lon_subset, lat_subset, precipitation_new.T, levels=levels[1:], cmap='jet')

    plt.colorbar(label='Precipitation (mm/hr)')
    plt.title(f'Precipitation {precip_file[11:13]}/{precip_file[9:11]}/{precip_file[5:9]} at {precip_file[14:16]}:00')

    # Add gridlines for latitude and longitude
    plt.grid(True, linestyle="dotted", linewidth=0.5, color="black")    
    
    plt.tight_layout()
    plt.show()


plot_precipitation(precip_files[150],
                   lon_min=start_longitude, lon_max=end_longitude, lat_min=start_latitude, lat_max=end_latitude)

We begin by plotting rainfall data. It accepts a specific precipitation file as input, enabling us to visualize the rainfall distribution for a particular time and date. The function extracts relevant data from the precipitation file, including latitude and longitude information, and subsets it to the desired geographical region. Contours of precipitation levels are superimposed on a map that distinguishes land from sea, providing context. The resulting plot showcases the spatial distribution of rainfall, crucial for identifying areas with high precipitation rates, which are indicative of cyclonic activity.



In [ ]:
def plot_slp(file_name, time_step, lon_min=-179, lon_max=179, lat_min=-49, lat_max=49):
    fn = slp_dir +file_name
    ds = nc.Dataset(fn)

    pressure = ds['SLP'][:]
    lat = ds['lat'][:]
    lon = ds['lon'][:]
    time = ds['time']

    time_index = time_step
    title_time = str(time_index).zfill(2)

    lon_indices = np.where((lon >= lon_min) & (lon <= lon_max))[0]
    lat_indices = np.where((lat >= lat_min) & (lat <= lat_max))[0]

    slp_time = pressure[time_index, lat_indices[0]:lat_indices[-1]+1, lon_indices[0]:lon_indices[-1]+1]

    # Plot the contour
    plt.figure(figsize=(12, 4))
    levels = np.linspace(np.min(slp_time), np.max(slp_time), 20)
    plt.contourf(lon[lon_indices], lat[lat_indices], slp_time, levels=levels[:], cmap='jet', 
                 extent=[lon_min, lon_max, lat_min, lat_max])
    plt.colorbar(label='Sea Level Pressure (Pa)')
    plt.title(f'Sea Level Pressure on {file_name[-10:-8]}/{file_name[-12:-10]}/{file_name[-16:-12]} at {title_time}:00')
    plt.grid(True, linestyle="dotted", linewidth=0.5, color='black')
    
    plt.show()

plot_slp(slp_files[18], 18,
         lon_min=start_longitude, lon_max=end_longitude, lat_min=start_latitude, lat_max=end_latitude)

Moving on to sea-level pressure data, this function takes a sea-level pressure file, a specific time step, and the geographical boundaries as input. It extracts relevant pressure data for the defined region and time step, generating a contour plot that illustrates variations in sea-level pressure. These pressure contours are vital for identifying low-pressure areas, which are indicative of potential cyclone formation. The contour levels and color mapping help visualize the pressure gradient across the region, offering insights into atmospheric conditions.

# Binary maps

In this section, we delve into the creation of binary maps from our meteorological datasets. These binary maps play a pivotal role in our cyclone tracking algorithm by simplifying the data and highlighting specific weather conditions that are indicative of cyclonic activity.

Before we proceed, it's important to establish the criteria for defining cyclonic conditions in our datasets. For precipitation data, we set a threshold of 10 mm/hr, meaning any region with rainfall exceeding this value will be marked as part of a potential cyclone. In the case of sea-level pressure, a threshold of 100,000 Pascal (Pa) is used. Regions with pressure values below this threshold are identified as potential cyclonic zones.

In [ ]:
threshold_pressure = 100000
threshold_rainfall = 10

In [ ]:
def delete_files_in_folder(folder_path):
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        file_list = os.listdir(folder_path)

        if file_list:
            for filename in file_list:
                file_path = os.path.join(folder_path, filename)

                if os.path.isfile(file_path):
                    os.remove(file_path)


## Creating Precipitation Binary Maps
To generate binary maps for precipitation, we extract the relevant precipitation data for the defined region, compares it to the threshold, and assigns binary values (1 for cyclonic conditions, 0 for non-cyclonic) to each grid point. These binary maps are saved as images, making them easily accessible for further analysis.

An example of the binary map is illustrated below for a specific date and time, providing insights into precipitation patterns during cyclonic events. The map showcases areas experiencing high rainfall (in black) and those outside the high rainfall influence (in white). Additionally, the grid structure and geographical coordinates are overlaid for reference, facilitating a comprehensive understanding of the data.

In [ ]:
def plot_precipitation_binary_sample(image_name, precip_file, threshold=8, lon_min=-179, lon_max=179, lat_min=-49, lat_max=49):
    
    fn = precip_dir + "/" + precip_file
    ds = nc.Dataset(fn)
    precipitation = ds["precipitation"][:]

    lat = ds["nlat"][:].data
    lon = ds["nlon"][:].data

    min_lon_index = np.abs(lon_min - lon).argmin()
    max_lon_index = np.abs(lon_max - lon).argmin()
    min_lat_index = np.abs(lat_min - lat).argmin()
    max_lat_index = np.abs(lat_max - lat).argmin()

    precipitation_new = precipitation[min_lon_index: max_lon_index, min_lat_index:max_lat_index]
    
    binary_map = np.where(precipitation_new > threshold, 1, 0)

    plt.figure(figsize=(12, 4))

    lon_subset = lon[min_lon_index: max_lon_index]
    lat_subset = lat[min_lat_index: max_lat_index]
    
        
    plt.contourf(lon_subset, lat_subset, binary_map.T, levels=[-0.5, 0.5, 1.5], colors=["white", "black"])
    
    plt.title(f'Precipitation {precip_file[11:13]}/{precip_file[9:11]}/{precip_file[5:9]} at {precip_file[14:16]}:00')
    plt.grid(True, linestyle="dotted", linewidth=0.5, color='black')
    plt.show() 
    

    
plot_precipitation_binary_sample(precip_files[150][5:16], precip_files[150], threshold=threshold_rainfall, 
                               lon_min=start_longitude, lon_max=end_longitude, lat_min=start_latitude, lat_max=end_latitude)
    

In [ ]:
def plot_precipitation_binary(image_name, precip_file, threshold=8, lon_min=-179, lon_max=179, lat_min=-49, lat_max=49):
    
    fn = precip_dir + "/" + precip_file
    ds = nc.Dataset(fn)
    precipitation = ds["precipitation"][:]

    lat = ds["nlat"][:].data
    lon = ds["nlon"][:].data

    min_lon_index = np.abs(lon_min - lon).argmin()
    max_lon_index = np.abs(lon_max - lon).argmin()
    min_lat_index = np.abs(lat_min - lat).argmin()
    max_lat_index = np.abs(lat_max - lat).argmin()

    precipitation_new = precipitation[min_lon_index: max_lon_index, min_lat_index:max_lat_index]
    
    binary_map = np.where(precipitation_new > threshold, 1, 0)

    plt.figure(figsize=(3.1, 1.56))

    lon_subset = lon[min_lon_index: max_lon_index]
    lat_subset = lat[min_lat_index: max_lat_index]
    
        
    plt.contourf(lon_subset, lat_subset, binary_map.T, levels=[-0.5, 0.5, 1.5], colors=["white", "black"])
    
    
    plt.axis('off')
    
    # Define the folder path
    folder_path = "/kaggle/working/precip/"

    # Create the folder if it doesn't exist
    os.makedirs(folder_path, exist_ok=True)

    plt.savefig(f"/kaggle/working/precip/{image_name}.png", pad_inches=0, dpi =100, bbox_inches='tight')
    plt.close()  # Close the figure
    

In [ ]:
delete_files_in_folder("/kaggle/working/precip")
for precip_file in precip_files:
    plot_precipitation_binary(precip_file[5:16], precip_file, threshold=threshold_rainfall, 
                               lon_min=start_longitude, lon_max=end_longitude, lat_min=start_latitude, lat_max=end_latitude)

## Creating Sea-Level Pressure Binary Maps
Similarly, for sea-level pressure data, we extract the pressure data for the specified region and time step, compares it to the threshold, and generates binary maps. These binary maps represent areas with low pressure, which are indicative of potential cyclonic activity.

As demonstrated below, the generated binary map displays the distribution of sea-level pressure anomalies for a given date and time. It provides valuable insights into areas with low pressure (here we are using a higher threshold just to show how it will look on a binary map), which are critical indicators for potential cyclonic behavior. Gridlines and geographical coordinates are overlaid on the map for reference, enhancing the understanding of the data.

In [ ]:
def plot_slp_binary_sample(slp_file, time_step, threshold, lon_min=-179, lon_max=179, lat_min=-49, lat_max=49):
    
    fn = slp_dir + slp_file
    ds = nc.Dataset(fn)

    pressure = ds['SLP'][:]
    lat = ds['lat'][:]
    lon = ds['lon'][:]
    time = ds['time']

    time_index = time_step
    title_time = str(time_index).zfill(2)


    lon_indices = np.where((lon >= lon_min) & (lon <= lon_max))[0]
    lat_indices = np.where((lat >= lat_min) & (lat <= lat_max))[0]

    slp_time = pressure[time_index, lat_indices[0]:lat_indices[-1]+1, lon_indices[0]:lon_indices[-1]+1]
    
    binary_map = np.where(slp_time < threshold, 1, 0)

    # Plot the contour
    plt.figure(figsize=(12, 4))
    levels = np.linspace(np.min(slp_time), np.max(slp_time), 20)
    plt.contourf(lon[lon_indices], lat[lat_indices], binary_map, levels=[-0.5, 0.5, 1.5], colors=["white", "black"], 
                 extent=[lon_min, lon_max, lat_min, lat_max])
    
    plt.title(f'Sea Level Pressure on {slp_file[-10:-8]}/{slp_file[-12:-10]}/{slp_file[-16:-12]} at {title_time}:00 (threshold = {threshold} Pa)')
    plt.grid(True, linestyle="dotted", linewidth=0.5, color='black')
    
    plt.show()

plot_slp_binary_sample(slp_files[18], 18, threshold = 100500,
         lon_min=start_longitude, lon_max=end_longitude, lat_min=start_latitude, lat_max=end_latitude)

In [ ]:
def plot_slp_binary(slp_file, time_step, threshold, lon_min=-179, lon_max=179, lat_min=-49, lat_max=49):
    
    fn = slp_dir + slp_file
    ds = nc.Dataset(fn)

    pressure = ds['SLP'][:]
    lat = ds['lat'][:]
    lon = ds['lon'][:]
    time = ds['time']

    time_index = time_step
    title_time = str(time_index).zfill(2)


    lon_indices = np.where((lon >= lon_min) & (lon <= lon_max))[0]
    lat_indices = np.where((lat >= lat_min) & (lat <= lat_max))[0]

    slp_time = pressure[time_index, lat_indices[0]:lat_indices[-1]+1, lon_indices[0]:lon_indices[-1]+1]
    
    binary_map = np.where(slp_time < threshold, 1, 0)

    # Plot the contour
    plt.figure(figsize=(3.1, 1.56))
    levels = np.linspace(np.min(slp_time), np.max(slp_time), 20)
    plt.contourf(lon[lon_indices], lat[lat_indices], binary_map, levels=[-0.5, 0.5, 1.5], colors=["white", "black"], 
                 extent=[lon_min, lon_max, lat_min, lat_max])
    
    
    plt.axis('off')

    save_dir = "/kaggle/working/slp/"
    os.makedirs(save_dir, exist_ok=True)
    plt.savefig(f"/kaggle/working/slp/{slp_file[-16:-8]}.{title_time}.png", 
                bbox_inches='tight', pad_inches=0, dpi=100)
    plt.close()  # Close the figure
#     plt.show()


In [ ]:
delete_files_in_folder("/kaggle/working/slp")

for slp_file in slp_files:
    for i in range(0, 24, 3):
        plot_slp_binary(slp_file,threshold=threshold_pressure, time_step = i, 
                         lon_min=start_longitude, lon_max=end_longitude, lat_min=start_latitude, lat_max=end_latitude)

In [ ]:
precip_binary_dir = "/kaggle/working/precip"
precip_images = sorted(os.listdir(precip_binary_dir))

In [ ]:
slp_binary_dir = "/kaggle/working/slp"
slp_images = sorted(os.listdir(slp_binary_dir))


Both sets of binary maps, one for precipitation and the other for sea-level pressure, are organized into separate directories. The precipitation binary maps are stored in the "precip" folder, while the sea-level pressure binary maps are saved in the "slp" folder. These maps serve as fundamental inputs for our cyclone tracking algorithm, allowing us to identify and track cyclones based on the defined criteria.

The binary maps streamline the data, making it easier to detect and monitor potential cyclonic activity, a critical step in our overall analysis of meteorological conditions for the year 2019.

# Tracking Algorithm

In this section, the tracking algorithm is elaborated, and its foundation relies on binary maps representing sea level pressure (SLP) and precipitation data generated beforehand. The code meticulously executes a series of steps aimed at the identification, analysis, and tracking of cyclones based on these crucial datasets.

Initially we have created functions for calculating distances and determining the total rainfall. These functions are integral to the subsequent processes. Following that, regions of interest are extracted from the binary images. These areas are characterised by heavy rainfall and low pressure zones. The images are then processed using edge detection techniques to identify contours. The total rainfall, centroid coordinates, and area within each contour are calculated and recorded. After recording the centroids, the tracking algorithm begins and step by step process is explained later.






In [ ]:
def calculate_distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [ ]:
def calculate_total_rainfall(precipitation_map, binary_map):
    # Calculate the total rainfall by summing up precipitation values where binary_map is 1
    total_rainfall = np.sum(precipitation_map * binary_map)
    return total_rainfall


In [ ]:
def find_centroids_area(image_name, image_path, first_day=False, rainfall=False):
    # Construct the full path to the image
    path = os.path.join(image_path, image_name) 
    
    # Read the image in grayscale
    first_image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    
    # Set the sigma value for Canny edge detection
    sigma = 7
    edges_first = cv2.Canny(first_image, sigma, sigma * 2)

    # Find the initial bounding boxes for objects in the image
    contours, _ = cv2.findContours(edges_first, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    regions = [cv2.boundingRect(contour) for contour in contours]
    
    # Initialize a list to store centroid and area information
    centroids_area = []
    
    # Extract the time step from the image name
    time_step = int(image_name[-6:-4])
    
    if rainfall:
        # Process precipitation data
        fn = precip_dir + f"3B42.{image_name[:-4]}.7.HDF.nc4"
        threshold = 10
        ds = nc.Dataset(fn)
        precipitation = ds["precipitation"][:]
        lat = ds["nlat"][:].data
        lon = ds["nlon"][:].data
        
        # Calculate the indices for latitude and longitude
        min_lon_index = np.abs(start_longitude - lon).argmin()
        max_lon_index = np.abs(end_longitude - lon).argmin()
        min_lat_index = np.abs(start_latitude - lat).argmin()
        max_lat_index = np.abs(end_latitude - lat).argmin()

        precipitation_new = precipitation[min_lon_index: max_lon_index, min_lat_index:max_lat_index]

        binary_map = np.where(precipitation_new > threshold, 1, 0)
        
        for region in regions:
            # Extract the bounding box coordinates
            x, y, w, h = region

            # Create a binary mask for the current contour
            mask = np.zeros_like(precipitation_new)
            mask[x:x+w, (120-y)-(h-1):(120-y)+1] = 1

            # Extract the corresponding region from the binary map
            contour_rainfall = binary_map * mask

            # Calculate total rainfall for the contour
            r = calculate_total_rainfall(precipitation_new, contour_rainfall)

            # Calculate the centroid of the bounding box
            centroid_x = x + w // 2
            centroid_y = y + h // 2
            area = w * h
            centroid_area = (centroid_x, centroid_y, area, r)
            centroids_area.append(centroid_area)
    
    else:
        # Process sea level pressure data
        fn = slp_dir + f"MERRA2_400.tavg1_2d_slv_Nx.{image_name[:-7]}.nc4.nc4"
        threshold = 100300
        
        ds = nc.Dataset(fn)
        pressure = ds['SLP'][:]
        lat = ds['lat'][:]
        lon = ds['lon'][:]
        time = ds['time']
        
        time_index = time_step
        
        lon_indices = np.where((lon >= start_longitude) & (lon <= end_longitude))[0]
        lat_indices = np.where((lat >= start_latitude) & (lat <= end_latitude))[0]

        
        slp_time = pressure[time_index, lat_indices[0]:lat_indices[-1]+1, lon_indices[0]:lon_indices[-1]+1]
        binary_map = np.where(slp_time < threshold, 1, 0)
        
        for region in regions:
            # Extract the bounding box coordinates
            x, y, w, h = region

            # Create a binary mask for the current contour
            mask = np.zeros_like(slp_time)
            mask[x:x+w, (120-y)-(h-1):(120-y)+1] = 1

            # Extract the corresponding region from the binary map
            contour_pressure = binary_map * mask

            # Calculate total rainfall for the contour
            r = calculate_total_rainfall(slp_time, contour_pressure)

            # Calculate the centroid of the bounding box
            centroid_x = x + w // 2
            centroid_y = y + h // 2
            area = w * h
            centroid_area = (centroid_x, centroid_y, area, r)
            centroids_area.append(centroid_area)
    
    if first_day:
        # Sort and select the top centroid based on area for the first day
        centroids_area.sort(key=lambda tup: tup[2], reverse=True)    
        centroids_area = centroids_area[:1]
    else:
        # Sort and select the top three centroids based on area for subsequent days
        centroids_area.sort(key=lambda tup: tup[2], reverse=True)    
        centroids_area = centroids_area[:3]
    
    return centroids_area


## Algorithm: Tracking Cyclones

This algorithm outlines a method for cyclone tracking by processing binary images of precipitation and SLP. Cyclones are characterized by regions of heavy rainfall and low-pressure centers. The algorithm identifies cyclone presence, records their tracks, and predicts their movement.

**Step 1: Initialization**

- Initialize an empty list to store cyclone tracks.

**Step 2: Data Processing**

- Loop through binary images of precipitation and pressure.

**Step 3: Identifying Cyclone Indicators**

- Find regions with high rainfall and low-pressure areas, indicative of cyclones.

**Step 4: Cyclone Presence Check**

- If no signs of a cyclone are detected in the data, proceed to step 5.

**Step 5: Recording Cyclone Tracks**

- If a cyclone was previously tracked, save its track information.
- Prepare to start tracking a new cyclone.

**Step 6: Detecting Cyclone**

- When indicators of a cyclone are detected, proceed to step 7.

**Step 7: First Day of Tracking**

- Calculate the distance between centroids of area detected in rainfall and pressure binary images.
- If they are close (within 20 units (pixels in this case)):
    - Mark the current date and time.
    - Begin recording the cyclone's path.
    - Note that this is not the first day of tracking.
- If they are far apart (more than 20 units (pixels in this case)), proceed to step 8.

**Step 8: Record Cyclone Track**

- If a cyclone was previously tracked, save its track information.
- Prepare to start tracking a new cyclone.

**Step 9: Tracking on Subsequent Days**

- On days following the first:
- Utilize data from the previous day to estimate the cyclone's location.
- Update the date and time.
- Identify the closest detected area that matches specific criteria:
    - If a suitable area is found:
        - Predict the cyclone's location for the next day.
        - Identify the closest low-pressure area (SLP).
        - Record the cyclone's track for the current day.

**Step 10: End of Data**

- After analyzing all available binary maps:
- If there is data from the last cyclone:
    - Save its track information.




   

In [ ]:
# Initialize dictionaries to store cyclone tracks
tracks = {}
path_cord = {}

# Initialize variables
first_day = True
time_stamp = None
t = 1

# Loop through binary images
for i, image_name in enumerate(slp_images):
    # Detect centroids of heavy precipitation areas
    centroids_area_precip = find_centroids_area(image_name, first_day=first_day, rainfall=True, image_path="/kaggle/working/precip/")
    
    # Detect centroids of low-pressure zones
    centroids_area_slp = find_centroids_area(image_name, first_day=first_day, image_path="/kaggle/working/slp/")
    
    # Check if centroids are available for either precipitation or SLP
    if (len(centroids_area_precip) == 0 or len(centroids_area_slp) == 0):
        # Cyclone disappeared
        if path_cord:
            # Save the current cyclone track information
            tracks[f"Track{t}"] = path_cord
            t += 1
        first_day = True
        path_cord = {}
    else:
        if first_day:
            # Calculate the distance between precipitation and SLP centroids
            dist = calculate_distance((centroids_area_precip[0][0], centroids_area_precip[0][1]),
                                      (centroids_area_slp[0][0], centroids_area_slp[0][1]))
            
            if dist < 20:
                # Mark the current date and time
                time_stamp = f"{image_name[4:6]}/{image_name[6:8]} {image_name[9:11]}:00"
                
                # Record cyclone track information
                path_cord[time_stamp] = [((centroids_area_precip[0][0] + centroids_area_slp[0][0]) / 2,
                                          (centroids_area_precip[0][1] + centroids_area_slp[0][1]) / 2,
                                          centroids_area_precip[0][2], centroids_area_precip[0][3]),
                                         centroids_area_precip[0],
                                         centroids_area_slp[0]]
                
                first_day = False
            else:
                # Distance between pressure and rainfall centroids is large, indicating a deviation
                if path_cord:
                    # Save the current cyclone track information
                    tracks[f"Track{t}"] = path_cord
                    t += 1
                first_day = True
                path_cord = {}
        else:
            # For subsequent days of tracking
            # Retrieve the coordinates of the cyclone from the previous day's tracking
            previous_day_cord = path_cord[time_stamp][0]
            time_stamp = f"{image_name[4:6]}/{image_name[6:8]} {image_name[9:11]}:00"
            
            min_dist = 20
            min_index = None
            for i, coord in enumerate(centroids_area_precip):
                # Calculate the distance between potential centroids and the previous day's coordinates
                distance = calculate_distance((coord[0], coord[1]), (previous_day_cord[0], previous_day_cord[1]))
                if (distance < min_dist) and (0.5 * previous_day_cord[2] < coord[2]):
                    min_dist = distance
                    min_index = i
            
            if min_index is None:
                next_day_cord_precip = previous_day_cord
            else:
                next_day_cord_precip = centroids_area_precip[min_index]
            
            # For Sea level pressure
            for i, coord in enumerate(centroids_area_slp):
                # Calculate the distance between pressure centroids and the updated precipitation coordinates
                distance = calculate_distance((coord[0], coord[1]), (next_day_cord_precip[0], next_day_cord_precip[1]))
                if distance < 20:
                    next_day_cord_slp = coord
                
            # Record cyclone track information for the current day
            path_cord[time_stamp] = [((next_day_cord_precip[0] + next_day_cord_slp[0]) / 2,
                                       (next_day_cord_precip[1] + next_day_cord_slp[1]) / 2,
                                       next_day_cord_precip[2], next_day_cord_precip[3]),
                                     next_day_cord_precip,
                                     next_day_cord_slp]
            
# Check if there is any remaining cyclone track information
if path_cord:
    # Save the final cyclone track information
    tracks[f"Track{t}"] = path_cord



We filtered the cyclone tracks to isolate those that align with the typical behavior of cyclones. This involves assessing the duration of each track, considering that our data is recorded every 3 hours. We select tracks that span from 3 to 15 days, a range commonly associated with cyclone lifespans. The chosen tracks are systematically labeled as 'Track1', 'Track2', and so on. 

In [ ]:
def filter_and_rename_tracks(track_data):
    filtered_tracks = {}
    track_counter = 1

    for track_name, data in track_data.items():
        if (320 > len(data) > 24):
            rainfall_area = [val[0][3] for key, val in data.items()]
            counts = Counter(rainfall_area)
            if max(counts.values()) < len(rainfall_area) / 2:
                filtered_tracks[f'Track{track_counter}'] = data
                track_counter += 1

    return filtered_tracks

filtered_track_data = filter_and_rename_tracks(tracks)

# Visualizing Cyclone Tracks

A visual representation of all the cyclone tracks are shown on a map. Each cyclone's movement is depicted on the map using a combination of rainfall and sea level pressure (SLP) coordinates.

* Cyclone tracks are coded for clarity: blue triangles rainfall, and 'x' markers denote SLP data.
* The paths of cyclones are shown as solid lines, illustrating their trajectories over time.
* Timestamps at the start and end of each track provide context about when these cyclones were active.
* Latitude and longitude markings are included on the map to help viewers understand the geographical locations.
In summary, this function provides a visual overview of cyclone movements, allowing viewers to see how they evolve over time and their interactions with land and sea.

In [ ]:
def plot_cyclone_paths(path_cord, tracks, lon_min, lon_max, lat_min, lat_max):
    plt.figure(figsize=(10, 4))

    ls_mask = nc.Dataset("/kaggle/input/trmm-precipitation/mask.nc4")

    # Get the variables from the netCDF file
    lon = ls_mask.variables["lon"][:]
    lat = ls_mask.variables["lat"][:]
    landseamask = ls_mask.variables["landseamask"][:]

    # Convert land-sea mask to binary representation (land=0, sea=1)
    landseamask_binary = np.where(landseamask == 100, 1, 0)

    # Find the corresponding indices for the specified latitude and longitude range
    lon_indices = np.where((lon >= lon_min) & (lon <= lon_max))[0]
    lat_indices = np.where((lat >= lat_min) & (lat <= lat_max))[0]

    # Subset the land-sea mask and corresponding latitude and longitude arrays
    landseamask_subset = landseamask_binary[lon_indices[0]:lon_indices[-1] + 1, lat_indices[0]:lat_indices[-1] + 1]

    plt.contourf(landseamask_subset.T, levels=[-0.5, 0.5, 1.5], colors=["papayawhip", "lightskyblue"],
                 extent=[0, 240, 0, 120])

    for track_name in tracks:
        track_data = path_cord.get(track_name, {})

        combined_coords = [coords[0] for coords in track_data.values()]
        rainfall_coords = [coords[1] for coords in track_data.values()]
        slp_coords = [coords[2] for coords in track_data.values()]

        # Add timestamps at the start and end of the cyclone's path
        start_timestamp = list(track_data.keys())[0]
        end_timestamp = list(track_data.keys())[-1]

#         plt.annotate(start_timestamp, (combined_coords[0][0], 120 - combined_coords[0][1]),
#                      textcoords="offset points", xytext=(20, 10), ha='center', fontsize=8, color='black')

#         plt.annotate(end_timestamp, (combined_coords[-1][0], 120 - combined_coords[-1][1]),
#                      textcoords="offset points", xytext=(20, 15), ha='center', fontsize=8, color='black')


        plt.scatter([coord[0] for coord in rainfall_coords], [120 - coord[1] for coord in rainfall_coords],
                    label=f"{track_name} - Rainfall", marker='^', s=10)

        plt.scatter([coord[0] for coord in slp_coords], [120 - coord[1] for coord in slp_coords],
                    label=f"{track_name} - Sea Level Pressure (SLP)", marker='x', s=10)

        plt.plot([coord[0] for coord in combined_coords], [120 - coord[1] for coord in combined_coords],
                 lw=1, markersize=2)
        

    # Changing axis ticks to lat lon values
    x_bars = np.arange(lon_min, lon_max + 1, 10)
    x_pos = np.linspace(0, 241, len(x_bars))
    plt.xticks(x_pos, x_bars)

    y_bars = np.arange(lat_min, lat_max + 1, 5)
    y_pos = np.linspace(0, 121, len(y_bars))
    plt.yticks(y_pos, y_bars)

    # Show the plot
#     plt.legend()
    plt.grid(True, linestyle="dotted", alpha=0.5, color="black")
    plt.title("Cyclone Tracks for 2019")
#     plt.savefig("all_path", dpi= 1000)


    plt.show()


track_names_to_plot = [track for track, val in filtered_track_data.items()]

plot_cyclone_paths(filtered_track_data, track_names_to_plot,
                   lon_min=start_longitude, lon_max=end_longitude, lat_min=start_latitude, lat_max=end_latitude)


# Visualizing Individual Cyclone Path and Intensity

The path and intensity of individual cyclone is shown below offering a detailed understanding of its movement in relation to land and sea areas.

* Cyclone paths are complemented by rainfall data (blue triangles) and sea level pressure (SLP) data (green 'x' markers). This comprehensive representation aids in comprehending the cyclone's interactions with these meteorological factors.
* Rainfall intensity is visualized as a filled area plot, highlighting daily average intensities. This presentation allows us to observe variations in rainfall intensity throughout the cyclone's track. A red dashed line signifies the mean rainfall intensity, serving as a valuable reference point.

Together, these functions empower users to explore the nuances of individual cyclone tracks, providing both geographical and meteorological context. This enhances our understanding of cyclone behavior and the impact of rainfall intensity during their journeys.

In [ ]:
def plot_cyclone_path(path_cord, track_name, lon_min, lon_max, lat_min, lat_max):    
    plt.figure(figsize=(10,4))

    ls_mask = nc.Dataset("/kaggle/input/trmm-precipitation/mask.nc4")

    # Get the variables from the netCDF file
    lon = ls_mask.variables["lon"][:]
    lat = ls_mask.variables["lat"][:]
    landseamask = ls_mask.variables["landseamask"][:]

    # Convert land-sea mask to binary representation (land=0, sea=1)
    landseamask_binary = np.where(landseamask == 100, 1, 0)


    # Find the corresponding indices for the specified latitude and longitude range
    lon_indices = np.where((lon >= lon_min) & (lon <= lon_max))[0]
    lat_indices = np.where((lat >= lat_min) & (lat <= lat_max))[0]

    # Subset the land-sea mask and corresponding latitude and longitude arrays
    landseamask_subset = landseamask_binary[lon_indices[0]:lon_indices[-1]+1, lat_indices[0]:lat_indices[-1]+1]


    plt.contourf(landseamask_subset.T, levels=[-0.5, 0.5, 1.5], colors=["papayawhip", "lightskyblue"],
                extent=[0,240,0,120])
    
    track_data = path_cord.get(track_name, {})
#     for track_data in path_cord.items():
    combined_coords = [coords[0] for coords in track_data.values()]
    rainfall_coords = [coords[1] for coords in track_data.values()]
    slp_coords = [coords[2] for coords in track_data.values()]
    
    # Add timestamps at the start and end of the cyclone's path
    start_timestamp = list(track_data.keys())[0]
    end_timestamp = list(track_data.keys())[-1]

    plt.annotate("start", (combined_coords[0][0], 120 - combined_coords[0][1]),
                 textcoords="offset points", xytext=(10,10), ha='center', fontsize=8, color='black')
    
    plt.annotate("end", (combined_coords[-1][0], 120 - combined_coords[-1][1]),
                 textcoords="offset points", xytext=(20,15), ha='center', fontsize=8, color='black')
    
    

    
    plt.scatter([coord[0] for coord in rainfall_coords], [120 - coord[1] for coord in rainfall_coords], 
                label="Rainfall", color='blue', marker='^', s=10)

    plt.scatter([coord[0] for coord in slp_coords], [120 - coord[1] for coord in slp_coords], 
                label="Sea Level Pressure (SLP)", color='green', marker="x", s=10)
    
    plt.plot([coord[0] for coord in combined_coords], [120 - coord[1] for coord in combined_coords], 
             label="Rainfall & SLP", lw=1, markersize=1, color='red')

    
    
   # Changing axis ticks to lat lon values
    x_bars = np.arange(lon_min, lon_max + 1, 10)
    x_pos = np.linspace(0, 241, len(x_bars)) 
    plt.xticks(x_pos, x_bars)

    y_bars = np.arange(lat_min, lat_max + 1, 5)
    y_pos = np.linspace(0, 121, len(y_bars)) 
    plt.yticks(y_pos, y_bars)

    
    # Show the plot
    plt.legend()
    plt.grid(True, linestyle="dotted", alpha = 0.5, color="black")
    plt.title(f"{track_name}: ({start_timestamp[3:5]}/{start_timestamp[:2]} to {end_timestamp[3:5]}/{end_timestamp[:2]}) ") 
    plt.show()


In [ ]:
def plot_rainfall_intensity(path_cord, track_name):
    
    track_data = path_cord.get(track_name, {})
    rainfall_daily_intensity = {}
    for timestamp, values in track_data.items():
        date = timestamp.split()[0]
        # Adjust the date format to "dd/mm" (day/month)
        date = "/".join(reversed(date.split("/")))
        
        intensity = values[0][3] / values[0][2]  # Calculate rainfall intensity (mm/hr)
        if date in rainfall_daily_intensity:
            rainfall_daily_intensity[date].append(intensity)
        else:
            rainfall_daily_intensity[date] = [intensity]

    rainfall_intensity = {date: sum(intensities) / len(intensities) for date, intensities in rainfall_daily_intensity.items()}

    dates = list(rainfall_intensity.keys())
    rainfall_intensity_values = list(rainfall_intensity.values())

    # Calculate the mean value of the daily averages
    mean_rainfall = np.mean(rainfall_intensity_values)

    plt.figure(figsize=(10, 4))
    plt.fill_between(dates, rainfall_intensity_values, alpha=0.5)
    plt.axhline(y=mean_rainfall, color='red', linestyle='--', label='Mean Rainfall')

    plt.title("Rainfall Intensity")
    plt.ylabel('Rainfall (mm/hr)')
    plt.legend()

    plt.show()


In [ ]:
for track, val in filtered_track_data.items():
    plot_cyclone_path(filtered_track_data, track,  
                       lon_min=start_longitude, lon_max=end_longitude, lat_min=start_latitude, lat_max=end_latitude)
    plot_rainfall_intensity(filtered_track_data, track)
    
    

# Conclusion

We may corroborate our findings by comparing the path of the cyclone traced by the algorithm to that determined by IBTrACS (International Best Track Archive for Climate Stewardship). IBTrACS is a widely recognized database that compiles historical cyclone tracks, making it a benchmark for tracking cyclones.

1. **Track 1:**
   - **Algorithm Prediction:** February 19 to February 24.
   - **Actual Occurrence:** Typhoon Wutip impacted the Philippines, Vietnam, and other areas from February 19 to February 26, 2019.

2. **Track 2:**
   - **Algorithm Prediction:** October 6 to October 12.
   - **Actual Occurrence:** Hagibis was a powerful typhoon that struck Japan from October 6 to October 13.

3. **Track 3:**
   - **Algorithm Prediction:** November 5 to November 12.
   - **Actual Occurrence:** Typhoon Halong developed east of the Northern Mariana Islands, with a duration from November 1 to November 8.

4. **Track 4:**
   - **Algorithm Prediction:** November 26 to December 4.
   - **Actual Occurrence:** Typhoon Kammuri, known in the Philippines as Typhoon Tisoy, was a powerful typhoon that impacted the Philippines in early December 2019. It formed on November 24, and dissipated on December 6.
   

The cyclone tracking algorithm uses a straightforward approach to determine the path of cyclones. It detects potential cyclonic activity by identifying regions of high rainfall and low pressure for each time frame. This approach successfully tracked the paths of various cyclones, including notable events such as Wutip, Hagibis, Halong, and Kammuri. We compared the cyclone paths generated by our algorithm, which incorporates both rainfall and pressure data, to those from IBTrACS, a well-known cyclone tracking database, to validate its performance. 

In addition to tracking the path of the cyclones, our algorithm provides invaluable insights into the affected geographical area and the intensity of the rainfall associated with the cyclones. These additional details can be critical in disaster preparedness and response strategies. Future improvements for the algorithm could include:
- Implementation of variable rainfall and pressure thresholds to fine-tune the algorithm's performance.
- Expanding the algorithm's application to a global scale, which has the potential to improve cyclone monitoring and prediction capabilities.